# Making A Neural Network From Scratch:

**Goal**: To implement a neural network using only numPy that has two hidden layers and one output layer with two activation functions. We can then train our neural network on the MNIST data-set, and test it to see our accuracy. 

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/mnist-digit-recognizer/train.csv
/kaggle/input/chinese-mnist-digit-recognizer/chineseMNIST.csv


I'm going to first import the mnist digits in csv form, every row represents an image and every column, a pixel, with the first column being the label of the number.

In [2]:
df = pd.read_csv("/kaggle/input/mnist-digit-recognizer/train.csv")
df.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [3]:
df.shape # rows show how many pictures there are in the csv 

(42000, 785)

We are going to convert this DataFrame into a np.array, and use test_train_split() by sklearn. Then we want to take the **tranpose** of each array so that each array is actually an individual picture (which will be one of the 784 nodes).

In [4]:
x_train, x_test, y_train, y_test = train_test_split(np.array(df.iloc[:, 1:]), np.array(df[['label']]), test_size = .4)
x_train, x_test, y_train, y_test = x_train.T, x_test.T, y_train.T, y_test.T 
pixels = df.shape[1] - 1
display(x_train[:, 0].shape, len(x_test[0]), x_train, y_train, y_test) # check configs
# n_xtrain (rows of pixels) = 784, m_xtrain (columns of items) = train/split

(784,)

16800

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

array([[2, 9, 3, ..., 9, 0, 9]])

array([[5, 9, 4, ..., 7, 3, 9]])

Above, we see that y_train is actually an array inside an array, we will fix that below:

In [5]:
y_train, y_test = y_train[0], y_test[0]
y_train, y_test

(array([2, 9, 3, ..., 9, 0, 9]), array([5, 9, 4, ..., 7, 3, 9]))

We also want the RGB values to be scaled from 0-1 so I will divide all training and test X values (the pixels) by 255. Without it some "gradient explosion" will happen. 

In [6]:
x_train, x_test = x_train / 255, x_test / 255

**Note**: Basically we want to make the train set an nxm matrix so we can left multiply it by a weight matrix to move from Rm -> R10 (10 is an arbitrary number for the first layer but makes computation easier). This will help to compute a linear combination with the weights and nodes 

# Initialization and Propagation: 
1. creating a random array of weights, W1, W2
2. creating a random array of biases, b1, b2
3. arrays for each layer:
    * input layer
    * first reLU layer (a function applied to each node)
    * second output layer (which will also have a softmax to find a probability distribution of numbers)
4. Make forward and backwards propagation. 

We are also going to start with random weights and biases. Below is a diagram in LaTeX explaining the matrix multiplcation **for a single picture**, where matrix $B$ is our input layer tranposed multiplied by our $A$ matrix, and the biases being added as matrix $C$. The result will be a $10 * m$ matrix, where $m$ is the number of training pictures.





$$\begin{bmatrix}
a_{11} & a_{12} & \cdots & a_{1,784} \\
a_{21} & a_{22} & \cdots & a_{2,784} \\
\vdots & \vdots & \ddots & \vdots \\
a_{10,1} & a_{10,2} & \cdots & a_{10,784}
\end{bmatrix}
\times
\begin{bmatrix}
b_{11} & b_{12} & \cdots & b_{1m} \\
b_{21} & b_{22} & \cdots & b_{2m} \\
\vdots & \vdots & \ddots & \vdots \\
b_{784,1} & b_{784,2} & \cdots & b_{784,m}
\end{bmatrix}
+
\begin{bmatrix}
c_{11} & c_{12} & \cdots & c_{1m} \\
c_{21} & c_{22} & \cdots & c_{2m} \\
\vdots & \vdots & \ddots & \vdots \\
c_{10,1} & c_{10,2} & \cdots & c_{10,m}
\end{bmatrix}
$$


Here are the softmax function and ReLU function definitions:

$
\text{softmax}(z_i) = \frac{e^{z_i}}{\sum_{j=1}^n e^{z_j}}
$

where $z_i$ is the i-th element of the input vector z. The softmax function will apply this to each $z_i$.

The softmax function is applied independently to each column of $z$. For a single column (logits for one image), softmax computes that equation.

$
\text{ReLU}(x) = \max(0, x)
$


In [7]:
def init_parameters(): 
    # populate a 10 x 784 matrix with random numbers, W1 is weight for first layer
    W1 = np.random.rand(10, pixels) -0.5
    # continue with biases, which should be a 10x1 vector, which we will broadcast
    # 10x1 -> 10xm so we can add the dot product + b
    b1 = np.random.rand(10, 1) - 0.5
    # second layer weights and biases
    W2 = np.random.rand(10, 10) - 0.5
    b2 = np.random.rand(10, 1) - 0.5
    # we ensure that the weights are centered around 0, instead of 0.5
    # we want some negative weights as well so that is why
    return W1, b1, W2, b2
    
def ReLU(Z): # return the same sized matrix, applied reLU
    return np.maximum(0,Z)
    
def SoftMax(Z): # input: 10 x m matrix
    # for numerical stability, we can add something to catch nan-values
    A = np.exp(Z) / sum(np.exp(Z))
    return A

    # 1. np.exp(Z) applied to every element in Z 
    # 2. np.sum(np.exp(Z)) sums all the values of Z from exp, scalar
    # 3. All elements are normalized, result = 10 x m, for each training result
def forward_propagation(W1, b1, W2, b2, X): # X is the initial input matrix
    # Z1 will be the resulting matrix from matrix multiplication
    Z1 = W1.dot(X) + b1 
    # apply reLU to change from linear combination -> non-linear function
    A1 = ReLU(Z1)
    Z2 = W2.dot(A1) + b2 # 10 x m
    A2 = SoftMax(Z2) # predictions
    # return values to update in backpropgation
    return Z1, A1, Z2, A2

## Back Propagation and Updating Weights
We will now be writing the backpropagation part: 
1. We need to write one hot, defined as Y (which is the vector equivalent of what the "actual" value is). This will help determine the cost 
2. Calculate how much each layer contributes to the cost function via partial derivatives:
   * dZ is the error of each of the columns (not the cost). This is written as the partial derivative of the cost with respect to the activation (second layer being soft-max and first layer being ReLU).
   *  dW is the change in cost with respect to the weights. 
   *  dB is the change in cost with respect to the biases.<br>
 For a softmax classifier, we'll use a cross-entropy loss function, which will basically come out to dZ if you take the derivative with respect to activation:

$$J(\hat{y}, y) = -\sum_{i=0}^{c} y_i \log(\hat{y}_i)$$

For the entire math behind what dz is, here is a forum: https://community.deeplearning.ai/t/calculating-gradient-of-softmax-function/1897

Here are the equations if the neural network was linked by one node at a time. z in this case is the the result of the linear combination of the weights + bias not related to "activation" ([3B1B's explanation](https://www.youtube.com/watch?v=Ilg3gGewQ5U&list=PLZHQObOWTQDNU6R1_67000Dx_ZCJB-3pi&index=3))
:

$\frac{\partial C_0}{\partial w^{(L)}} = \frac{\partial z^{(L)}}{\partial w^{(L)}} \frac{\partial a^{(L)}}{\partial z^{(L)}} \frac{\partial C_0}{\partial a^{(L)}}$

$z^{(L)} = w^{(L)} a^{(L-1)} + b^{(L)}$ <br>
$a^{(L)} = \sigma(z^{(L)})$ <br>
$C^{(0)} = a^{(L)} - y$

Notice how the new layer is based off activation result of the previous layer, this can be also translated into matricies in code you will see below.  

By terrible convention, these names dZ, dB, and dW are equal to the following in 3b1b language: 

$dZ = \frac{\partial C_0}{\partial a^{(L)}}$,
$dB = \frac{\partial C_0}{\partial b^{(L)}}$,
$dW = \frac{\partial C_0}{\partial w^{(L)}}$,


3. Using these equations, we can compute a gradient, which for each layer is a vector that tells us the mangitutde of the effect (cost) of each partial derivative (fancy wording for what we just had above).
   
4. We will then adjust our weights by subtracting each layer's weights by their specific dW, each layer's biases by their dB. We cannot specifically change the previous layer's (L-1) activation, but used it in helping compute each dW and dB. Here's what it will look like:

    $w^{(1)} = w^{(1)} - a * dw^{(1)}$<br>
    $b^{(1)} = b^{(1)} - a * db^{(1)}$<br>
   $w^{(2)} = w^{(2)} - a * dw^{(2)}$<br>
    $b^{(2)} = w^{(2)} - a * db^{(2)}$<br>

This is how gradient descent will work, as dw and db will converge to 0 through **minimizing the cost** with respect to all its partial derivitives. Alpha is the learning rate (a)



* After all these initialization steps, we will be able to run our own neural network by repitition. 

In [8]:
def one_hot(Y): # input: the example labels, output: all the one hot Y's
    one_hot_Y = np.zeros((Y.size, Y.max() + 1)) 
    # (#examples, 0-9 *10), we will take transpose later
    
    one_hot_Y[np.arange(Y.size), Y] = 1
    # basically you can put arrays inside index selection in 2d arrays, which is super neat
    # this looks like one_hot_Y[(0 - 784 pixels), (4,5,1,2,4... to m images)]
    return one_hot_Y.T
    
def derivative_ReLU(Z): 
    return Z > 0 # boolean matrix works since True = 1
    
def back_propagation(Z1, A1, Z2, A2, W2, X, Y): # figuring out partial derivatives
    # Z1, Z2 = result of a linear combination of weights + biases
    # A1, A2, = result of activation being called on Z1, Z2
    # X = training input, Y = ouput results
    # reminder that Z1 - A2 are all 10 x m matricies and will broadcast Y.
    m, _ = df.shape # number examples
    
    dZ2 = A2 - one_hot(Y) 
    # dZ2 = partial d of the change in cost in terms of soft-max activation
    # Note this equation is based off of the error because it is the last layer
  
    dW2 = 1/m * dZ2.dot(A1.T)
    # partial d of the second layer costs to weights
    # 1/m is applied to every value and is not part of the p derivative
    # transpose ensures that matrix multiplication exists
   
    dB2 = 1/m * np.sum(dZ2, axis = 1, keepdims = True)
    # collapses all rows into 10 x m -> 10 x 1, 
    # partial d of the second layer costs to biases 
    # also has 1/m applied since this matrix is applied to all training examples
    
    dZ1 = W2.T.dot(dZ2) * derivative_ReLU(Z1)
    # the W2.T activation transpose serves to "go backwards" in a sense
    # mathmatical proof will for all calculations will be linked below.

    dW1 = 1/m * dZ1.dot(X.T)
    dB1 = 1/m * np.sum(dZ1, axis = 1, keepdims = True)
    # these are actually the same formula as before due to the chain rule !
    return dW1, dB1, dW2, dB2 

def adjust_parameters(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha): 
    W1 = W1 - alpha * dW1
    b1 = b1 - alpha * db1    
    W2 = W2 - alpha * dW2  
    b2 = b2 - alpha * db2   
    return W1, b1, W2, b2

# Training / Testing the Neural Network
We will now be implmeneting the training functionality of the network and test for its accuracy. With gradient descent, the idea is to forward -> backward propagate -> adjust parameters (weights and biases) and then do it again with those new weights and biases. 

In [9]:
def get_predictions(A2):
    return np.argmax(A2, axis = 0) 
    # argmax for a 10 x m array (A2)
    # 
    # second activation layer is the result of SoftMax

def accuracy(predictions, Y):
    print(predictions, Y)
    return np.sum(predictions == Y) / Y.size
    # thank you Data 8 for encoding this formula into my head

def train_gradient_descent(X, Y, alpha, iterations):
    W1, b1, W2, b2 = init_parameters()
    #print("init: ", W1, b1, W2, b2)
    for i in range(iterations): 
        Z1, A1, Z2, A2 = forward_propagation(W1, b1, W2, b2, X)
        #print("forward: ", Z1, A1, Z2, A2) *testing
        #print(A2.shape)
        dW1, db1, dW2, db2  = back_propagation(Z1, A1, Z2, A2, W2, X, Y)
        #print("back: ", dW1, db1, dW2, db2)
        W1, b1, W2, b2 = adjust_parameters(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha)
        #print("adj: ", W1, b1, W2, b2)
        if i % 20 == 0: 
            predictions = get_predictions(A2)
            print(f"Iteration {i} : {accuracy(predictions, Y)}")
    return W1, b1, W2, b2 

W1, b1, W2, b2 = train_gradient_descent(x_train, y_train, 0.1, 500)


[1 5 4 ... 5 1 8] [2 9 3 ... 9 0 9]
Iteration 0 : 0.12563492063492063
[1 0 4 ... 0 0 4] [2 9 3 ... 9 0 9]
Iteration 20 : 0.18162698412698414
[1 0 3 ... 0 0 4] [2 9 3 ... 9 0 9]
Iteration 40 : 0.23793650793650795
[1 0 3 ... 0 0 3] [2 9 3 ... 9 0 9]
Iteration 60 : 0.2811111111111111
[1 0 3 ... 0 0 3] [2 9 3 ... 9 0 9]
Iteration 80 : 0.31746031746031744
[1 0 3 ... 0 0 3] [2 9 3 ... 9 0 9]
Iteration 100 : 0.36138888888888887
[1 0 3 ... 5 0 3] [2 9 3 ... 9 0 9]
Iteration 120 : 0.4083333333333333
[1 0 3 ... 5 0 3] [2 9 3 ... 9 0 9]
Iteration 140 : 0.45361111111111113
[1 9 3 ... 5 0 3] [2 9 3 ... 9 0 9]
Iteration 160 : 0.498452380952381
[1 9 3 ... 9 0 3] [2 9 3 ... 9 0 9]
Iteration 180 : 0.5488095238095239
[1 9 3 ... 9 0 3] [2 9 3 ... 9 0 9]
Iteration 200 : 0.5962301587301587
[1 9 3 ... 9 0 3] [2 9 3 ... 9 0 9]
Iteration 220 : 0.6350396825396826
[1 9 3 ... 9 0 3] [2 9 3 ... 9 0 9]
Iteration 240 : 0.6684126984126985
[1 9 3 ... 9 0 3] [2 9 3 ... 9 0 9]
Iteration 260 : 0.6918650793650793
[1 9 3 